In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.cluster import KMeans
from imblearn.under_sampling import EditedNearestNeighbours
import tensorflow as tf
from tensorflow.keras.optimizers import legacy
from tensorflow.keras import layers, Model
import pickle

In [2]:
with open('/content/drive/MyDrive/nsl_kdd_preprocessed.pkl', 'rb') as file:
    X_train, X_test, y_train, y_test = pickle.load(file)

In [3]:
print("Kích thước của tập huấn luyện (X_train):", X_train.shape)
print("Kích thước của tập kiểm tra (X_test):", X_test.shape)
print("Kích thước của tập huấn luyện (y_train):", y_train.shape)
print("Kích thước của tập kiểm tra (y_test):", y_test.shape)

Kích thước của tập huấn luyện (X_train): (125972, 41)
Kích thước của tập kiểm tra (X_test): (22543, 41)
Kích thước của tập huấn luyện (y_train): (125972,)
Kích thước của tập kiểm tra (y_test): (22543,)


In [4]:
# Thống kê số mẫu mỗi lớp
y_train.value_counts()

attack
normal             67342
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: count, dtype: int64

In [5]:
y_test.value_counts()

attack
normal             9711
neptune            4656
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint               319
mailbomb            293
snmpgetattack       178
portsweep           157
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail             14
rootkit              13
xterm                13
teardrop             12
xlock                 9
land                  7
xsnoop                4
ftp_write             3
worm                  2
loadmodule            2
perl                  2
sqlattack             2
udpstorm              2
phf                   2
imap                  1
Name: count, dtype: int64

In [6]:
t = 20000 # Maximum number in a label
# Lay danh sach cac class thuoc minority va majority
Smin = []
Smaj = []

for label, cnt in y_train.value_counts().items():
  if cnt >= (t * 0.7): # trainset:testset = 7:3
    Smaj.append(label)
  else:
    Smin.append(label)

print(Smin)
print(Smaj)

['satan', 'ipsweep', 'portsweep', 'smurf', 'nmap', 'back', 'teardrop', 'warezclient', 'pod', 'guess_passwd', 'buffer_overflow', 'warezmaster', 'land', 'imap', 'rootkit', 'loadmodule', 'ftp_write', 'multihop', 'phf', 'perl', 'spy']
['normal', 'neptune']


In [7]:
# Số lượng cụm và tổng số mẫu cần lấy từ mỗi lớp
n_clusters = 23 # Num of classes
samples_needed_per_class = int(t * 0.7) # t * 0.7

# Khởi tạo một list để lưu các cụm và nhãn sau khi xử lý
selected_samples_X = []
selected_samples_y = []

# Lọc ra các mẫu dữ liệu chỉ thuộc các nhãn trong Smaj
majority_mask = np.isin(y_train, Smaj)
X_majority = X_train[majority_mask]
y_majority = y_train[majority_mask]

print('ENN')
# Áp dụng ENN để loại bỏ nhiễu
enn = EditedNearestNeighbours()
X_enn, y_enn = enn.fit_resample(X_majority, y_majority)

# Duyệt qua từng nhãn trong nhãn đã làm sạch
for label in Smaj:
    print('Label: ', label)
    label_mask = y_enn == label
    X_label = X_enn[label_mask]

    # Áp dụng k-means trên dữ liệu đã làm sạch cho mỗi nhãn
    if len(X_label) > 1:  # Kiểm tra có đủ dữ liệu để phân cụm không
        print('Clustering')
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        kmeans.fit(X_label)
        labels = kmeans.labels_

        # Tính số mẫu cần lấy từ mỗi cụm
        cluster_counts = np.bincount(labels, minlength=n_clusters)
        cluster_ratios = cluster_counts / cluster_counts.sum()
        samples_per_cluster = np.floor(cluster_ratios * samples_needed_per_class).astype(int)

        # Đảm bảo rằng tổng số mẫu đúng bằng 16000, điều chỉnh nếu cần
        while samples_per_cluster.sum() < samples_needed_per_class:
            print('not enough')
            samples_per_cluster[np.argmax(cluster_counts - samples_per_cluster)] += 1

        print('Compressing')
        # Lấy mẫu từ mỗi cụm
        for cluster in range(n_clusters):
            print('Cluster ', cluster)

            cluster_mask = labels == cluster
            X_cluster = X_label[cluster_mask]

            if len(X_cluster) > 0:

                # Chọn ngẫu nhiên các mẫu từ cụm
                if len(X_cluster) > samples_per_cluster[cluster]:
                    indices = np.random.choice(len(X_cluster), samples_per_cluster[cluster], replace=False)
                    selected_samples_X.append(X_cluster.iloc[indices])
                    selected_samples_y.extend([label] * samples_per_cluster[cluster])
                else:
                    selected_samples_X.append(X_cluster)
                    selected_samples_y.extend([label] * len(X_cluster))

# Tạo tập dữ liệu cuối cùng từ các mẫu đã chọn
X_train_balanced = np.vstack(selected_samples_X)
y_train_balanced = np.array(selected_samples_y)

print("Số lượng mẫu cuối cùng được chọn: ", X_train_balanced.shape[0])

ENN
Label:  normal
Clustering


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


not enough
not enough
not enough
not enough
not enough
not enough
not enough
not enough
not enough
not enough
Compressing
Cluster  0
Cluster  1
Cluster  2
Cluster  3
Cluster  4
Cluster  5
Cluster  6
Cluster  7
Cluster  8
Cluster  9
Cluster  10
Cluster  11
Cluster  12
Cluster  13
Cluster  14
Cluster  15
Cluster  16
Cluster  17
Cluster  18
Cluster  19
Cluster  20
Cluster  21
Cluster  22
Label:  neptune
Clustering


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


not enough
not enough
not enough
not enough
not enough
not enough
not enough
not enough
not enough
not enough
not enough
Compressing
Cluster  0
Cluster  1
Cluster  2
Cluster  3
Cluster  4
Cluster  5
Cluster  6
Cluster  7
Cluster  8
Cluster  9
Cluster  10
Cluster  11
Cluster  12
Cluster  13
Cluster  14
Cluster  15
Cluster  16
Cluster  17
Cluster  18
Cluster  19
Cluster  20
Cluster  21
Cluster  22
Số lượng mẫu cuối cùng được chọn:  28000


In [8]:
unique_values, counts = np.unique(y_train_balanced, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

neptune: 14000
normal: 14000


In [9]:
# Khai báo các hằng số
BATCH_SIZE = 32
LATENT_DIM = 128
NUM_EPOCHS = 10
SAMPLES_TO_GENERATE = int(t * 0.7)

# Định nghĩa mô hình Generator
def build_generator():
    noise = layers.Input(shape=(LATENT_DIM,))
    x = layers.Dense(256, activation="relu")(noise)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(np.prod(X_train.shape[1:]), activation=None)(x)
    samp = layers.Reshape(X_train.shape[1:])(x)
    return Model(noise, samp)

# Định nghĩa mô hình Discriminator
def build_discriminator():
    samp = layers.Input(shape=X_train.shape[1:])
    x = layers.Flatten()(samp)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(1)(x)
    return Model(samp, x)


# Hàm để sinh dữ liệu
def generate_data_for_class(label):
    global X_train_balanced, y_train_balanced, X_train, y_train
    current_data = X_train[y_train == label]
    current_samples = current_data.shape[0]
    samples_needed = SAMPLES_TO_GENERATE - current_samples

    X_train_balanced = np.concatenate([X_train_balanced, current_data], axis=0)
    y_train_balanced = np.concatenate([y_train_balanced, np.full(current_samples, label)], axis=0)

    if samples_needed > 0:
        noise = np.random.normal(0, 1, (samples_needed, LATENT_DIM))
        generated_samples = generator.predict(noise)

        # Gộp dữ liệu sinh ra vào X_train và y_train
        X_train_balanced = np.concatenate([X_train_balanced, generated_samples], axis=0)
        y_train_balanced = np.concatenate([y_train_balanced, np.full(samples_needed, label)], axis=0)

# Huấn luyện WGAN cho mỗi class thiểu số
for label in Smin:

    # Tối ưu hoá và loss function
    optimizer = legacy.RMSprop(lr=0.00005)

    generator = build_generator()
    discriminator = build_discriminator()

    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    discriminator.trainable = False

    noise = layers.Input(shape=(LATENT_DIM,))
    samp = generator(noise)
    validity = discriminator(samp)
    combined = Model(noise, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    data = X_train[y_train == label]
    valid = np.ones((BATCH_SIZE, 1))
    fake = -np.ones((BATCH_SIZE, 1))

    for epoch in range(NUM_EPOCHS):
        idx = np.random.randint(0, data.shape[0], BATCH_SIZE)
        real_samp = data.iloc[idx]

        noise = np.random.normal(0, 1, (BATCH_SIZE, LATENT_DIM))
        gen_samp = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_samp, valid)
        d_loss_fake = discriminator.train_on_batch(gen_samp, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = combined.train_on_batch(noise, valid)

    generate_data_for_class(label)  # Sinh dữ liệu cho class hiện tại

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


324/324 [==============================] - 1s 4ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


326/326 [==============================] - 2s 5ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

438/438 [==============================] - 2s 4ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


438/438 [==============================] - 3s 6ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


438/438 [==============================] - 2s 4ms/step


In [10]:
unique_values, counts = np.unique(y_train_balanced, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

back: 14000
buffer_overflow: 14000
ftp_write: 14000
guess_passwd: 14000
imap: 14000
ipsweep: 14000
land: 14000
loadmodule: 14000
multihop: 14000
neptune: 14000
nmap: 14000
normal: 14000
perl: 14000
phf: 14000
pod: 14000
portsweep: 14000
rootkit: 14000
satan: 14000
smurf: 14000
spy: 14000
teardrop: 14000
warezclient: 14000
warezmaster: 14000


In [11]:
# Giam du lieu X_test, y_test bang cach chon ngau nhien

def reduce_data(X_test, y_test, n):
    unique_classes = np.unique(y_test)
    X_test_reduced = np.empty((0, X_test.shape[1]))
    y_test_reduced = np.array([])

    for label in unique_classes:
        indices = np.where(y_test == label)[0]
        if len(indices) > n:
            indices = np.random.choice(indices, n, replace=False)
        X_subset = X_test.iloc[indices]
        y_subset = y_test.iloc[indices]

        X_test_reduced = np.vstack((X_test_reduced, X_subset))
        y_test_reduced = np.concatenate((y_test_reduced, y_subset))

    return X_test_reduced, y_test_reduced

X_test_reduced, y_test_reduced = reduce_data(X_test, y_test, int(t * 0.3))

In [12]:
unique_values, counts = np.unique(y_test_reduced, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

apache2: 737
back: 359
buffer_overflow: 20
ftp_write: 3
guess_passwd: 1231
httptunnel: 133
imap: 1
ipsweep: 141
land: 7
loadmodule: 2
mailbomb: 293
mscan: 996
multihop: 18
named: 17
neptune: 4656
nmap: 73
normal: 6000
perl: 2
phf: 2
pod: 41
portsweep: 157
processtable: 685
ps: 15
rootkit: 13
saint: 319
satan: 735
sendmail: 14
smurf: 665
snmpgetattack: 178
snmpguess: 331
sqlattack: 2
teardrop: 12
udpstorm: 2
warezmaster: 944
worm: 2
xlock: 9
xsnoop: 4
xterm: 13


In [15]:
# Lưu dữ liệu vào file
data = {
    'X_train': X_train_balanced,
    'X_test': X_test_reduced,
    'y_train': y_train_balanced,
    'y_test': y_test_reduced
}

with open('nsl-kdd-balanced.pkl', 'wb') as file:
    pickle.dump(data, file)